In [1]:
import pandas as pd
import numpy as np

# Load full CSV
df = pd.read_csv("../data/nutrition_filtered.csv", usecols=[0, 1], header=0, names=["dish_id", "calories"])

# Load train/test dish_ids
with open("../data/train_filtered.txt") as f:
    train_ids = set(line.strip() for line in f)

with open("../data/test_filtered.txt") as f:
    test_ids = set(line.strip() for line in f)

#print(df["dish_id"].head(3))
#print(list(train_ids)[:5])

# Filter based on dish_id column (which is named "dish_id")
train_df = df[df["dish_id"].isin(train_ids)].copy()
test_df  = df[df["dish_id"].isin(test_ids)].copy()

# Add image path column
train_df["image_path"] = train_df["dish_id"].apply(lambda x: f"../data/images/{x}.png")
#print(train_df["calories"].head(5))
test_df["image_path"]  = test_df["dish_id"].apply(lambda x: f"../data/images/{x}.png")


In [2]:
import tensorflow as tf

IMG_SIZE = (224, 224)

def preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, 0.1)
    img = tf.image.random_contrast(img, 0.9, 1.1)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0  # normalize
    return img, label

def make_dataset(df, batch_size=16, shuffle=True):
    paths = df["image_path"].values
    labels = df["calories"].values.astype("float32")

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# create train and validation datasets
train_ds = make_dataset(train_df, batch_size=16, shuffle=True)
val_ds = make_dataset(test_df, batch_size=16, shuffle=False)

In [ ]:
# 1. Imports for model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model, Input, optimizers, regularizers

# 2. Load pretrained MobileNetV2 (no top, global-avg pooling)
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
base_model.trainable = False

# 3. Add your regression head
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1)(x)  # one scalar output

model = Model(inputs, outputs)

# 4. Compile
model.compile(
    optimizer=optimizers.Adam(),
    loss='mse',
    metrics=['mae']
)

# 5. Quick summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,651,969 (10.12 MB)

 Trainable params: 393,985 (1.50 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [4]:
from sklearn.metrics import mean_absolute_error, r2_score

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='mae', patience=4, restore_best_weights=True)
#    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=2, min_lr=1e-6)
]

# 1. Train (no intermediate validation)
history = model.fit(
    train_ds,
    epochs=10
)

# 2. Get predictions + true labels from val_ds
y_true = []
y_pred = []

for batch_imgs, batch_labels in val_ds:
    preds = model(batch_imgs, training=False)        # shape (batch,1)
    y_true.extend(batch_labels.numpy().flatten())   # actual calories
    y_pred.extend(preds.numpy().flatten())          # predicted calories

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 3. Metrics
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"Final validation MAE: {mae:.2f} kcal")
print(f"Final validation R²: {r2:.3f}")

Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 61s 209ms/step - loss: 58712.6133 - mae: 174.6781
Epoch 2/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 44s 179ms/step - loss: 25369.4492 - mae: 104.4419
Epoch 3/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 45s 180ms/step - loss: 21993.4531 - mae: 96.0853
Epoch 4/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 45s 180ms/step - loss: 18417.4570 - mae: 92.9111
Epoch 5/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 43s 179ms/step - loss: 16154.6562 - mae: 87.2700
Epoch 6/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 43s 181ms/step - loss: 16768.4492 - mae: 88.8757
Epoch 7/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 43s 180ms/step - loss: 12841.9707 - mae: 81.7858
Epoch 8/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 43s 181ms/step - loss: 14292.6074 - mae: 80.9809
Epoch 9/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 43s 181ms/step - loss: 15171.0703 - mae: 79.1023
Epoch 10/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 43s 180ms/step - loss: 14389.8135 - mae: 78.0853
Final validation MAE: 88.64 kcal
Final validation R²: 0.664
